In [1]:
import data as da

# additional package sklearn + pandas + plot + seaborn

### 1 dataset
The data set was read using the data.py file and broken up and sliced into training and test sets, normalised using z-socre, and noisy using PCA to reduce and raise the dimensionality.

In [2]:
data_pro=da.Data()
data=data_pro.read_data()
xs,ys=data_pro.split_x_y(data)
xs_trans, xs_pca = data_pro.data_noise(10,xs)
xs, xs_train, xs_test, ys_train, ys_test = data_pro.data_processing(xs_pca,ys,True)

print('training set shape: ',xs_train.shape)
print('test set shape:     ', xs_test.shape)

training set shape:  (3918, 11)
test set shape:      (980, 11)


### 2 KNN
For the training set data use the KNeighborsRegressor(), a k-nearest neighbour (KNN) algorithm in sklearn.
#### 2.1 find best parameter
using GridSearchCV() in sklearn, which use cross-validated method,to determine the optimal parameters in knn method, where the search parameters (param_grid) are set as the assignment parameters (weights), k-values (n_neighbors) and distance calculation methods (metric), and n_neighbors is set to an odd number from 1-69<br/>
Output the best parameters and scores for the training set fit

In [27]:
# 2. knn
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# 2.1 find best hyper-parameter values; k-fold on training set
param_grid = [{
    'weights': ["uniform", "distance"],
    'n_neighbors': range(1,70,2),
    'metric':['euclidean', 'manhattan', 'cosine']
}]

knn_reg = KNeighborsRegressor()
knn_search = GridSearchCV(knn_reg, param_grid,n_jobs=-1)
knn_search.fit(xs_train, ys_train)

print("The best parameters is",knn_search.best_params_)
print('score of parameter is',knn_search.best_score_)


The best parameters is {'metric': 'cosine', 'n_neighbors': 41, 'weights': 'distance'}
score of parameter is 0.5009061479085019


#### 2.2 test dataset
Assign the optimal parameters to KNeighborsRegressor() as above, fit (.fit) predict (.predict) to the test and training sets, and output its fit score.

In [3]:
# 2.2 use best parameter on test set
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor(metric='cosine', n_neighbors=41, weights='distance')

knn_reg.fit(xs_train, ys_train)

ys_train_pred = knn_reg.predict(xs_train)
ys_test_pred = knn_reg.predict(xs_test)

#2.3 fit result
import fit_result
fit_result.score('KNN', ys_train, ys_train_pred,ys_test, ys_test_pred)


Train score of KNN  trian set 1.0
Test score of KNN  test set 0.4659363083084658 

Train mean error of KNN  trian set 0.0
Test mean error of KNN  test set 0.4535910741600085 

Train R2 of KNN  trian set 1.0
Test R2 score of KNN  test set 0.45905417857011865


### 3 SVM
Since the linear kernal operation for a support vector machine (SVM) using GridSearchCV() takes a very long time, this program tries to run it for 6 hours using 7 parameters and did not get results.<br/>
The program divides the linear (linear) and non-linear Gaussian kernel functions (rbf) of the SVM into two separate parts for Grid Search and compares the scores to select the optimal parameters and kernel functions, where kernal rbf uses SVR() and kernal linear uses LinearSVR(). The latter linear kernel function algorithm is more efficient than the libsvr-based SVR algorithm and can scale almost linearly to millions of samples and features [https://scikit-learn.org/stable/modules/svm.html#complexity (1.4.4)].<br/>
Special, more noisy observationsshould decrease C value, decreasing C corresponds to more regularization, and larger C values will take more time to train, sometimes up to 10 times longer.
GridSearchCV() in the last two paragraphs sets n_jobs=-1 to use all processors and increase the speed of computing.

In [31]:
# 3 SVM
# 3.1 find best parameter
# 3.1.1 find best hyper-parameter C values for SVM(kernal=rbf)
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

parameters = [
    {
        'C': [0.001,0.01,0.1,1,10,100,1000,float("inf")],
        'tol':[0.001,0.01,0.1,1,10],
        'epsilon':[0.01,0.1,1,10]
    }
]

svc_search = GridSearchCV(SVR(kernel='rbf'),parameters,n_jobs=-1) 
svc_search.fit(xs_train, ys_train)

print("The best parameters is",svc_search.best_params_)
print('R2 score of parameter is',svc_search.best_score_)

The best parameters is {'C': 1, 'epsilon': 0.1, 'tol': 0.1}
R2 score of parameter is 0.3791132691035906


In [34]:
# 3.1.2 use LinearSVR to find the best hyper-parameter C values for SVR (kernal=linear)
from sklearn.svm import LinearSVR

parameters = [
    {
        'C': [0.0001,0.001,0.1,1,10,100,1000,float("inf")],
        'tol':[0.001,0.01,0.1,1,10],
        'epsilon':[0.01,0.1,1,10]
    }
]

svc_search = GridSearchCV(LinearSVR(),parameters,n_jobs=-1)
svc_search.fit(xs_train, ys_train)

print("The best parameters is",svc_search.best_params_)
print('score of parameter is',svc_search.best_score_)


The best parameters is {'C': 1, 'epsilon': 0.01, 'tol': 0.001}
score of parameter is 0.27021640529488894


Comparing the optimal parameters and scores of the two methods, we found that for the SVM method kernel="rbf", C=1 gives a better fit, so we assigns SVC() to the test and training sets and performs the fitting prediction and score calculation.

In [4]:
# 3.2 use best parameter on test set
# 通过score of parameter看出rbf比linear的预测效果更好
from sklearn.svm import SVR

svr_reg = SVR(kernel="rbf", C=1, tol=0.1, epsilon=0.1)

svr_fit = svr_reg.fit(xs_train, ys_train)

ys_train_pred = svr_fit.predict(xs_train)
ys_test_pred = svr_fit.predict(xs_test)

# 3.3 result of svm
import fit_result
fit_result.score('SVM', ys_train, ys_train_pred,ys_test, ys_test_pred)


Train score of SVM  trian set 0.4919518716978365
Test score of SVM  test set 0.37427014450047025 

Train mean error of SVM  trian set 0.4502472119109347
Test mean error of SVM  test set 0.5608808544606592 

Train R2 of SVM  trian set 0.4905883459751398
Test R2 score of SVM  test set 0.37405297171928886
